<a href="https://colab.research.google.com/github/hshuai97/competition/blob/main/%E8%A5%BF%E9%97%A8%E5%AD%90_AI%E6%99%BA%E8%83%BD%E9%97%AE%E7%AD%94%E5%BA%94%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

比赛网址：[Siemens03](https://www.biendata.xyz/competition/Siemens03/leaderboard/)

代码参考：[link1](https://towardsdatascience.com/question-answering-with-pretrained-transformers-using-pytorch-c3e7a44b4012)

huggingface预训练好的中文QA模型：[link2](https://huggingface.co/models?pipeline_tag=question-answering&sort=downloads)

# 安装库

In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))

try:
  import transformers
except ModuleNotFoundError:
  !pip install transformers


from transformers import pipeline
import os
import pandas as pd
import time



# 读测试数据

In [ ]:
# 查看数据
path = '/content/drive/MyDrive/Colab_Notebooks/comp/data-siemens/'

data = pd.read_csv(os.path.join(path, 'sample_sub.tsv'), sep='\t')
data

,id,answer
0,1,物联网（I
1,2,物联网概念
2,3,1999年
3,4,同年，在美
4,5,同年，在美
...,...,...
1501,1502,我国税收体
1502,1503,改革开放以
1503,1504,食品制造企
1504,1505,创新销售模


In [ ]:
test_data = pd.read_csv(os.path.join(path, 'test.tsv'), sep='\t')
test_data

,Unnamed: 0,id,paragraph,question
0,0,1,物联网（Internet of Things，简称IoT）是指通过各种信息传感器、射频识别技...,物联网是通过各种装置与技术，怎样采集任何需要的物体或过程？
1,1,2,物联网概念最早出现于比尔盖茨1995年《未来之路》一书，在《未来之路》中，比尔盖茨已经提及物...,1998年，美国麻省理工学院创造性地提出了什么？
2,2,3,1999年，美国Auto-ID首先提出“物联网”的概念，主要是建立在物品编码、RFID技术和...,1999年中科院建立了什么？
3,3,4,同年，在美国召开的移动计算和网络国际会议提出了，“传感网是下一个世纪人类面临的又一个发展机遇...,美国的移动计算和网络国际会议提出了什么？
4,4,5,同年，在美国召开的移动计算和网络国际会议提出了，“传感网是下一个世纪人类面临的又一个发展机遇...,美国《技术评论》提出未来改变人们生活的十大技术之首将是什么？。
...,...,...,...,...
1501,1501,1502,我国税收体制改革的原则是“简税制，宽税基，低税率，严征管”。税率是衡量税负轻重的重要指标，代...,我国税收体制改革的原则是什么？
1502,1502,1503,改革开放以来，中国经济始终保持着高速增长，但这种增长是以要素驱动和投资驱动为中心的粗放式增长...,改革开放以来，中国经济始终保持着高速增长，但这种增长是怎样增长？
1503,1503,1504,食品制造企业需要设备和技术上的创新。企业应积极引入先进设备， 在实现进项税额抵扣的同时，可以...,食品制造企业需要什么上的创新？
1504,1504,1505,创新销售模式，拓宽销路。食品产品单位价值较低，保质期较短的特点，决定了销售环节对于食品制造业...,食品产品什么的特点，决定了销售环节对于食品制造业来说尤为重要？


# 预测答案并保存

In [ ]:

t0 = time.time()
question_answering = pipeline("question-answering", model="mrm8488/bert-multi-cased-finetuned-xquadv1", tokenizer="mrm8488/bert-multi-cased-finetuned-xquadv1")

id = []
answer = []

for i in range(len(test_data['id'])):
  t_id  = test_data['id'].iloc[i]
  t_context = test_data['paragraph'].iloc[i]
  t_ques =  test_data['question'].iloc[i]
  result = question_answering(question=t_ques, context=t_context)  # 预测答案

  id.append(t_id)
  answer.append(result['answer'])

# 保存答案
df = pd.DataFrame({'id': id, 'answer': answer})


df.to_csv(os.path.join(path, 'res.tsv'),sep='\t',  index=False)

print(f'total time: {(time.time()-t0)/60:.4f}mins')

total time: 22.2353mins


In [ ]:
res_data = pd.read_csv(os.path.join(path, 'res.tsv'), sep='\t')
res_data

,id,answer
0,1,监控、连接、互动
1,2,物联网
2,3,传感网
3,4,“传感网
4,5,传感网络技术
...,...,...
1501,1502,“简税制
1502,1503,粗放式
1503,1504,设备和技术
1504,1505,食品产品
